In [ ]:
# 1. 필수 라이브러리 설치
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch

from torch.utils.data import Dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from ydata_profiling import ProfileReport
from sklearn.utils.class_weight import compute_class_weight

# 허깅페이스에 있는 라이브러리를 사용하기 위한 선언
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch import nn


# 2. 데이터 로드 함수 (폴더 구조를 타고 들어가 모든 JSON을 읽음)
def load_legal_data(base_path):
    data_list = []
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('.json'):
                with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                    try:
                        content = json.load(f)
                        info = content.get('info', {})
                        label = content.get('label', {})
                        data_list.append({
                            'category': info.get('lawClass', ''),    # 법률 분류 코드 (예: 02)
                            'case_num': info.get('caseNum', ''),     # 사건 번호
                            'case_name': info.get('caseName', ''),    # 사건명
                            'case_code': info.get('caseCode', ''),    # 사건 코드 (헌마, 헌바 등)
                            'question': label.get('input', ''),       # 질의 내용 (추가 권장)
                            'body': label.get('output', '')           # 응답 내용
                        })
                    except Exception as e:
                        print(f"Error reading {file}: {e}")
                        continue
    return pd.DataFrame(data_list)

# 3. 데이터 불러오기 및 중간 확인
train_df = load_legal_data('./dataset/training')
test_df = load_legal_data('./dataset/haksup')

print(f"학습 데이터: {len(train_df)}건 / 검증 데이터: {len(test_df)}건")
print(train_df.columns)

train_df.head() # 현재 컬럼 형태와 데이터 샘플 출력

KeyboardInterrupt: 

In [ ]:
def label_case_type(row):
    text = f"{row['case_name']} {row['question']} {row['body']}"

    # 1️⃣ 헌법재판소 사건
    if row['case_code'].startswith('헌'):
        return '헌법'

    # 2️⃣ 형사소송
    criminal_keywords = [
        '사기', '폭행', '상해', '살인', '강도', '절도',
        '처벌', '징역', '벌금', '구속', '기소', '피고인', '유죄'
    ]
    if any(k in text for k in criminal_keywords):
        return '형사소송'

    # 3️⃣ 가사소송
    family_keywords = [
        '이혼', '양육권', '친권', '위자료', '재산분할', '혼인'
    ]
    if any(k in text for k in family_keywords):
        return '가사소송'

    # 4️⃣ 행정소송
    admin_keywords = [
        '처분', '취소', '과세', '부과', '행정', '허가',
        '등록', '공무원', '징계'
    ]
    if any(k in text for k in admin_keywords):
        return '행정소송'

    # 5️⃣ 민사소송
    civil_keywords = [
        '손해배상', '계약', '채무', '채권',
        '임대차', '소유권', '부당이득'
    ]
    if any(k in text for k in civil_keywords):
        return '민사소송'

    return '기타'

train_df['case_type'] = train_df.apply(label_case_type, axis=1)
test_df['case_type'] = test_df.apply(label_case_type, axis=1)

print(train_df[['case_name', 'case_code', 'case_type']].head())

                     case_name case_code case_type
0       의료보험법 제55조 등 에 대한 헌법소원        헌마        헌법
1  공정거래위원회고시 제92의1호 제5조 등 위헌확인        헌마        헌법
2                     기소유예처분취소        헌마        헌법
3                 고소사건진정종결처분취소        헌마        헌법
4          검사의 공소권행사 에 관한 헌법소원        헌마        헌법


In [ ]:
# 학습 데이터에 대한 상세 리포트 생성
# 데이터에 빈 곳은 없는지, 어떤 단어가 많이 나오는지 리포트로 확인합니다.
profile = ProfileReport(train_df, title="Legal AI Training Data Report")
profile.to_notebook_iframe() # 주피터 노트북 화면에 바로 출력

# 3. 'row_report.html'이라는 이름으로 파일 저장
profile.to_file("row_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:17<00:00,  2.50s/it]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# 리포트 내용 여기에서 확인 'Overview' / 'Variables'
# 전체적인 요약 정보와 데이터 타입 확인
train_df.info()


# 결측치(누락되거나 비어있는값) 개수 상세 확인
train_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   category   80000 non-null  object
 1   case_num   80000 non-null  object
 2   case_name  80000 non-null  object
 3   case_code  80000 non-null  object
 4   question   80000 non-null  object
 5   body       80000 non-null  object
 6   case_type  80000 non-null  object
dtypes: object(7)
memory usage: 4.3+ MB


category     0
case_num     0
case_name    0
case_code    0
question     0
body         0
case_type    0
dtype: int64

In [ ]:
# 통계치 및 분포 확인 / 'Statistics' 및 'Histogram' 
# 수치형 데이터의 기술 통계량 (평균, 4분위수 등)
train_df.describe()

,category,case_num,case_name,case_code,question,body,case_type
count,80000,80000,80000,80000,80000,80000,80000
unique,1,33193,8178,10,46904,79759,6
top,02,,,,,건축물관리법 제51조 제2항 규정에 따르면 무기 또는 1년 이상의 징역에 처합니다.,형사소송
freq,80000,35026,35026,69941,32566,11,37831


In [ ]:
# case_code가 빈 값인 데이터들만 모아서 case_type을 확인
empty_code_df = train_df[train_df['case_code'].str.strip() == ""]
print("사건코드가 없는 데이터의 소송 유형 분포:")
print(empty_code_df['case_type'].value_counts())

사건코드가 없는 데이터의 소송 유형 분포:
case_type
형사소송    37798
기타      21723
행정소송     9379
민사소송      967
가사소송       74
Name: count, dtype: int64


In [ ]:
# 범주형(텍스트) 데이터의 빈도수 확인
print(f"[Category Total: {len(train_df['category']):,}]")
print(train_df['category'].value_counts().apply(lambda x: f"{x:,}"), '\n')  #소송 분류 확인

# 컬럼명이 'law' 또는 'article' 등으로 되어 있을 수 있습니다.
print(f"[Case Type Total: {train_df['case_type'].count():,}]")
print(train_df['case_type'].value_counts().head(10).apply(lambda x: f"{x:,}"), '\n')
# 관련법률 빈도 / 상위 10개만 확인 

print(f"[Case Code Total: {train_df['case_code'].count():,}]")
print(train_df['case_code'].value_counts().head(10).apply(lambda x: f"{x:,}"), '\n')
 #승소/패소 균형(분포)확인

# 사건명 빈도 확인
train_df['case_code'] = train_df['case_code'].replace(['', ' '], '일반법원')

print(f"[Case Code Total: {train_df['case_code'].count():,}]")
print(train_df['case_code'].value_counts().head(10).apply(lambda x: f"{x:,}"), '\n')


# 데이터 컬럼 이름 확인 
print(train_df.columns)

[Category Total: 80,000]
category
02    80,000
Name: count, dtype: object 

[Case Type Total: 80,000]
case_type
형사소송    37,831
기타      21,767
헌법       9,940
행정소송     9,421
민사소송       967
가사소송        74
Name: count, dtype: object 

[Case Code Total: 80,000]
case_code
      69,941
헌마     5,460
헌가     2,856
헌바       970
헌아       546
헌사       105
각하       104
기각        13
헌라         3
기타         2
Name: count, dtype: object 

[Case Code Total: 80,000]
case_code
일반법원    69,941
헌마       5,460
헌가       2,856
헌바         970
헌아         546
헌사         105
각하         104
기각          13
헌라           3
기타           2
Name: count, dtype: object 

Index(['category', 'case_num', 'case_name', 'case_code', 'question', 'body',
       'case_type'],
      dtype='object')


In [ ]:
# 1. NaN(결측치)이거나 공백만 있는 행을 모두 찾습니다.
empty_code_df = train_df[train_df['case_code'].isna() | (train_df['case_code'].astype(str).str.strip() == "")]

if not empty_code_df.empty:
    print("사건코드가 없는 데이터의 소송 유형 분포:")
    print(empty_code_df['case_type'].value_counts())
else:
    print("확인 결과: 사건코드가 비어 있는 데이터가 하나도 없습니다!")
    
# 전체 행 중 중복된 데이터 개수
train_df.duplicated().sum()

# 법률 본문 중복확인
train_df['body'].duplicated().sum()

# 판결문이 같은 사건이 꽤 있음. -> 실제 판례라는 증거.

확인 결과: 사건코드가 비어 있는 데이터가 하나도 없습니다!


np.int64(241)

In [ ]:
# 특정 컬럼(예: 법률 본문/TF-IDF / BERT에서 쓸 통합 텍스트)의 중복 확인
train_df['legal_text'] = (
    train_df['case_name'] + " " +
    train_df['question'] + " " +
    train_df['body']
)

train_df['legal_text'].duplicated().sum()
# 중복 데이터 57개 개념을 더 확실히 익히게 만드는 것

np.int64(57)

In [ ]:
# 수치형 컬럼이 2개 미만이므로 상관관계 분석생략하고 
#TF-IDF(어휘빈도의 머신러닝)방식 -> KLUE-BERT(딥러닝) 진행.
# TR-IDF는 TF 단어빈도 하나의 문서 내에서 얼마나 자주 등장하는지
# IDF 전체 문서들중에서 얼마나 드물게 등장하는지

# TF-IDF(핵심단어 뽑기) + Logistic(예측하기) & 문장길이 기반 max_length 산정
# 텍스트를 수치화 -> 데이터 분포를 통해 최적의 길이 정하기.


# 이게바로 데이터 전처리!!!!
# TF-IDF는 단어의 빈도를 계산할때
# 핵심 단어를 쉽게 뽑기 위해 하나의 문장으로 만들어 문서 전체의 맥락을 반영.
train_df['text'] = (
    train_df['case_name'] + " " +
    train_df['question'] + " " +
    train_df['body']
)

test_df['text'] = (
    test_df['case_name'] + " " +
    test_df['question'] + " " +
    test_df['body']
)


# 머신러닝 시작! - 로지스틱 회귀
# (이진 분류, 회귀-> 로지스틱 회귀 모델은 학습을 통해 단어별로 점수(가중치)를 매김)
# 1-1. 문장 길이 기반 max_length 산정 (합친 텍스트 데이터 사용)
def calculate_max_length(texts):
    
    # 단어(공백 기준) 개수 계산
    lengths = [len(text.split()) for text in texts]
    
    # 전체 데이터의 95%를 커버하는 길이를 계산
    max_len = int(np.percentile(lengths, 99.9))

    print(f"추천 max_length (99.9% 커버리지): {max_len}")
    return max_len


# 1-2. TF-IDF + Logistic Regression
# 실제 데이터 적용
max_len_val = calculate_max_length(train_df['text'])

tfidf = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf.fit_transform(train_df['text'])
y_train = train_df['case_type'] 


lr_model = LogisticRegression(max_iter=1000, class_weight='balanced')
lr_model.fit(X_tfidf, y_train)


추천 max_length (99.9% 커버리지): 449


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


머신러닝
TF-IDF를 활용하여 Logistic Regression모델을 사용.
→ 단어에 점수를 매겨서 다 더한 후 높은 점수 소송타입으로 분류

분류되어있는 데이터셋의 텍스트 통합 후 단어 분리 및 파악

In [ ]:
tfidf_model = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=30000,
        ngram_range=(1, 2),
        min_df=3
    )),
    ('clf', LogisticRegression(
        max_iter=1000,
        class_weight='balanced'  # 🔥 불균형 대응
    ))
])

tfidf_model.fit(train_df['text'], train_df['case_type'])
preds = tfidf_model.predict(test_df['text'])

print(classification_report(test_df['case_type'], preds))
# precision(정밀도), recall(재현율), f1-score(조회 평균), Support (실제 데이터갯수)

              precision    recall  f1-score   support

        가사소송       0.31      0.50      0.38        10
          기타       0.89      0.93      0.91      3332
        민사소송       0.40      0.70      0.51        81
        행정소송       0.81      0.73      0.77       801
          헌법       0.95      0.99      0.97      1234
        형사소송       0.98      0.94      0.96      4542

    accuracy                           0.92     10000
   macro avg       0.72      0.80      0.75     10000
weighted avg       0.93      0.92      0.92     10000



In [ ]:
# 딥러닝 시작! KLUE-BERT모델(한국어 사전학습 언어 모델/ BERT가 모델임- 문맥 양방향 이해)

# 트랜스포머: 딥러닝 모델의 구조(아키텍처) AI모델의 기반이 됨.(정보해석:인코더/결과물제작 디코더)
#  ㄴBERT: 트랜스포머 구조의 언어모델
# 허깅페이스: 딥러닝 모델 플랫폼/라이브러리 제공/데이터셋 제공
# 3-1. 토크나이저 불러오기 (이 부분이 빠져서 에러가 났습니다!)
model_name = "klue/bert-base" #허깅페이스에서 가져올 모델이름
tokenizer = AutoTokenizer.from_pretrained(model_name) #허깅페이스에서 제공하는 토크나이저 로더


# 1. 라벨을 숫자로 변환하고 매핑 정보 보관
train_df['case_type'] = train_df['case_type'].astype('category')
num_labels = len(train_df['case_type'].cat.categories) # 카테고리 개수 (여기서는 6)
labels = train_df['case_type'].cat.codes.tolist()

# 2. 클래스 가중치 계산 (데이터 불균형 해결)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# 3. 모델 불러오기 (num_labels 수정) 허깅페이스에서 제공하는 모델불러오는거야.
model = AutoModelForSequenceClassification.from_pretrained( 
    "klue/bert-base", 
    num_labels=num_labels
).to(device)

# 4. 토큰화 (이제 에러 없이 실행됩니다)
train_encodings = tokenizer(
    train_df['text'].tolist(),
    truncation=True,
    padding=True,
    max_length=512
)  # 1,625개 중 512개까지만 사용 (BERT의 한계)


# 결과를 눈으로 확인하기 위한 코드
print(f"1. 클래스 개수: {num_labels}")
print(f"2. 계산된 가중치: {class_weights}")
print(f"3. 토큰화된 데이터 구조: {train_encodings.keys()}")
print(f"4. 첫 번째 문장의 토큰화 결과(일부): {train_encodings['input_ids'][0][:10]}")

OSError: 이 작업을 완료하기 위한 페이징 파일이 너무 작습니다. (os error 1455)

In [ ]:
# 2-1. PyTorch용 데이터셋 클래스 정의 (필수 단계)
class LegalDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# 2-2. 데이터셋 생성 (라벨은 반드시 0, 1 숫자여야 함)
train_dataset = LegalDataset(train_encodings, train_df['case_type'].tolist())
test_dataset = LegalDataset(test_encodings, test_df['case_type'].tolist())

# 2-3. 커스텀 Trainer 정의 (불균형 데이터 대응 가중치 적용)
from transformers import Trainer

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # 이전에 계산하신 class_weights를 GPU/CPU로 보냅니다.
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# 2-4. Trainer 가동
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset  # 검증에 테스트 데이터 사용
)

trainer.train() # 실제 학습 시작!
